In [ ]:
import torch
import ultralytics
print(f"Torch version: {torch.__version__}")
print(f"YOLO version: {ultralytics.__version__}")
print(f"GPU Available: {torch.cuda.is_available()}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Torch version: 2.5.1+cu121
YOLO version: 8.3.241
GPU Available: True


In [1]:
import pandas as pd

# Load the raw files we downloaded earlier via wget
ui_details = pd.read_csv("ui_details.csv")
app_details = pd.read_csv("app_details.csv")

# Merge on 'App Package Name' to connect UI Numbers to Categories
merged = pd.merge(ui_details, app_details, on='App Package Name')

# Save the exact file your Harvester is looking for
merged.to_csv("merged_industry_map.csv", index=False)

print("✅ 'merged_industry_map.csv' created successfully!")
print(f"Sample mapping: {merged[['UI Number', 'Category']].head(3)}")

✅ 'merged_industry_map.csv' created successfully!
Sample mapping:    UI Number                 Category
0          0                  Medical
1          1                  Medical
2          2  Video Players & Editors


In [ ]:
import pandas as pd
import os
import cv2
from ultralytics import YOLO
from tqdm import tqdm
import torch

# --- CORRECTED PATHS ---
MODEL_PATH = "best.pt"
RICO_IMAGE_DIR = "combined_images/combined"  # Pointing to the nested 'combined' folder
WEB_IMAGE_DIR = "web_data_raw"
METADATA_PATH = "merged_industry_map.csv"
OUTPUT_BASE = "Expert_Library"
CONF_THRESHOLD = 0.5
MAX_RICO_PER_CAT = 1500 

# --- INITIALIZE YOLO ---
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = YOLO(MODEL_PATH).to(device)
print(f"---  AI Harvester Active on {device} ---")

# PROCESS MOBILE UI IMAGES
def process_mobile():
    print("--- Phase 1: Processing Rico (Mobile) ---")
    df = pd.read_csv(METADATA_PATH, low_memory=False)
    cat_counts = {}

    for _, row in tqdm(df.iterrows(), total=len(df)):
        category = str(row['Category']).replace(" ", "_").replace("&", "")
        if cat_counts.get(category, 0) >= MAX_RICO_PER_CAT: continue
        
        ui_id = row['UI Number']
        # Rico images in unique_uis are usually .jpg
        img_path = os.path.join(RICO_IMAGE_DIR, f"{ui_id}.jpg")
        if not os.path.exists(img_path):
            img_path = img_path.replace(".jpg", ".png")
            if not os.path.exists(img_path): continue

        results = model.predict(source=img_path, imgsz=1024, conf=CONF_THRESHOLD, verbose=False)
        img = cv2.imread(img_path)
        if img is None: continue
        
        # Cropping each detected element
        saved = False
        for r in results:
            for i, box in enumerate(r.boxes):
                label = model.names[int(box.cls[0])]
                target_dir = os.path.join(OUTPUT_BASE, category, "Mobile", label)
                os.makedirs(target_dir, exist_ok=True)
                
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                crop = img[y1:y2, x1:x2]
                if crop is not None and crop.size > 0:
                    cv2.imwrite(os.path.join(target_dir, f"m_{ui_id}_{i}.jpg"), crop)
                    saved = True
        if saved: cat_counts[category] = cat_counts.get(category, 0) + 1
# PROCESS WEB UI IMAGES
def process_web():
    print("--- Phase 2: Processing YashJain (Web) ---")
    for subset in ['train', 'val', 'test']:
        # Pointing to the 'images' subfolder inside each split
        subset_img_path = os.path.join(WEB_IMAGE_DIR, subset, "images")
        if not os.path.exists(subset_img_path): continue
        
        images = [f for f in os.listdir(subset_img_path) if f.lower().endswith(('.png', '.jpg'))]
        for img_name in tqdm(images, desc=f"Web {subset}"):
            img_path = os.path.join(subset_img_path, img_name)
            
            results = model.predict(source=img_path, imgsz=1280, conf=CONF_THRESHOLD, verbose=False)
            img = cv2.imread(img_path)
            if img is None: continue
            
            # Crop and save web UI components
            
            for i, box in enumerate(results[0].boxes):
                label = model.names[int(box.cls[0])]
                target_dir = os.path.join(OUTPUT_BASE, "Web_General", "Web", label)
                os.makedirs(target_dir, exist_ok=True)
                
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                crop = img[y1:y2, x1:x2]
                if crop is not None and crop.size > 0:
                    cv2.imwrite(os.path.join(target_dir, f"w_{img_name}_{i}.jpg"), crop)

if __name__ == "__main__":
    process_mobile()
    process_web()
    print("---  HARVEST COMPLETE! ---")


    #This code prepares your dataset by detecting, cropping, and organizing UI elements from mobile and web screenshots so that later they can be scored for visual quality and style  saved to Expert_Library .

---  AI Harvester Active on cuda:0 ---
--- Phase 1: Processing Rico (Mobile) ---


100%|██████████| 66261/66261 [24:25<00:00, 45.20it/s] 


--- Phase 2: Processing YashJain (Web) ---


Web test: 100%|██████████| 76/76 [00:05<00:00, 13.80it/s]

---  HARVEST COMPLETE! ---


In [9]:
import faiss
res = faiss.StandardGpuResources()  

AttributeError: module 'faiss' has no attribute 'StandardGpuResources'

In [ ]:
import torch
import clip
from PIL import Image
import os
import faiss
import numpy as np
from tqdm import tqdm

# 1. Load CLIP "Style Judge" (Using GPU for the math, even if FAISS is CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)


EXPERT_DIR = "Expert_Library"
index_file = "expert_style_index.bin"
paths_file = "image_paths.txt"

def build_style_index():
    all_embeddings = [] # will store CLIP vectors for all expert images
    image_paths = [] # will store the file paths
    
    # Walks through every folder in Expert_Library,Collects all image file paths,Prepare the list of images to convert to vectors
    print(" Starting Style Vectorization ---")
    for root, dirs, files in os.walk(EXPERT_DIR):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(os.path.join(root, file))

    if not image_paths:
        print(" No images found in Expert_Library! Did the harvest fail?")
        return

    # Process in batches for GPU efficiency
    batch_size = 64
    for i in tqdm(range(0, len(image_paths), batch_size)):
        batch_paths = image_paths[i:i+batch_size]
        batch_imgs = []
        
        #Preprocess and convert images to CLIP vectors
        for p in batch_paths:
            try:
                img = preprocess(Image.open(p)).unsqueeze(0)
                batch_imgs.append(img)
            except:
                continue
            
        if not batch_imgs: continue

        #Encode images with CLIP
        with torch.no_grad():
            img_stack = torch.cat(batch_imgs).to(device)
            embeddings = model.encode_image(img_stack)
            embeddings /= embeddings.norm(dim=-1, keepdim=True)
            all_embeddings.append(embeddings.cpu().numpy())

    # Create FAISS Index
    embeddings_np = np.vstack(all_embeddings).astype('float32')
    dimension = embeddings_np.shape[1] # 512 for CLIP
    
    # index is standard for similarity
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings_np)
    
    # 4. Save to Disk
    faiss.write_index(index, index_file)
    with open(paths_file, "w") as f:
        for p in image_paths:
            f.write(p + "\n")
            
    print(f" Brain Built! Indexed {len(image_paths)} professional components into '{index_file}'.")

if __name__ == "__main__":
    build_style_index()

    

# Summary:

# Collects all expert UI images

# Converts them into numeric vectors using CLIP

# Stores vectors in FAISS index for fast similarity search

# Saves paths to know which vector belongs to which image

 Starting Style Vectorization ---
 No images found in Expert_Library! Did the harvest fail?


In [ ]:
import torch
import clip
import faiss
import numpy as np
import cv2
from PIL import Image
from ultralytics import YOLO

# --- Load Models & Index ---
device = "cuda" if torch.cuda.is_available() else "cpu"
yolo_model = YOLO("best.pt").to(device)
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)
index = faiss.read_index("expert_style_index.bin")

def generate_final_audit(image_path, output_path="final_audit_report.jpg"):
    img = cv2.imread(image_path)
    if img is None: return print(" Image not found")

    # 1. Detection
    results = yolo_model.predict(image_path, conf=0.4, imgsz=1024, verbose=False)
    scores = []

    # 2. Component Analysis
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = yolo_model.names[int(box.cls[0])]
            
            # Crop & Style Vector
            crop = img[y1:y2, x1:x2]
            if crop.size == 0: continue
            pil_crop = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
            img_input = clip_preprocess(pil_crop).unsqueeze(0).to(device)
            
            with torch.no_grad():
                user_vec = clip_model.encode_image(img_input)
                user_vec /= user_vec.norm(dim=-1, keepdim=True)
                
            # Search Similarity
            distances, _ = index.search(user_vec.cpu().numpy().astype('float32'), 1)
            sim_score = max(0, min(100, 100 - (distances[0][0] * 45)))
            scores.append(sim_score)

            # Draw Detection Box & Label
            color = (0, 255, 0) if sim_score > 75 else (0, 255, 255) if sim_score > 50 else (0, 0, 255)
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
            cv2.putText(img, f"{label}: {sim_score:.1f}%", (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # 3. Final Overall Scoring
    overall_avg = np.mean(scores) if scores else 0
    
    # Create Black Header Bar
    overlay = img.copy()
    cv2.rectangle(overlay, (0, 0), (img.shape[1], 80), (0, 0, 0), -1)
    cv2.addWeighted(overlay, 0.7, img, 0.3, 0, img)
    
    # Burn Overall Score into Image
    status = "EXCELLENT" if overall_avg > 85 else "GOOD" if overall_avg > 70 else "NEEDS WORK"
    header_text = f"OVERALL SIMILARITY SCORE: {overall_avg:.1f}% ({status})"
    cv2.putText(img, header_text, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

    # Save Result
    cv2.imwrite(output_path, img)
    print(f" Audit Finished. Overall Similarity: {overall_avg:.1f}%")
    print(f" Marked Image Saved: {output_path}")

# --- Run ---
generate_final_audit("test_ui.png")



 Audit Finished. Overall Similarity: 86.1%
 Marked Image Saved: final_audit_report.jpg
